In [137]:
# imports
import numpy as np ; import re ; import pandas as pd

# load files
PWD = "/Users/antongolles/Documents/Work/ITU-Research/DBS_textual/"
relative_location = "../data/Shareholder_decks/"
date_latest = '2022-05-18'

file = np.load(relative_location+f'text_from_shareholder_decks_{date_latest}.npz', allow_pickle=True)
text_dict = file[file.files[0]].item()

file = np.load('../data/keywords_dict.npz', allow_pickle=1)
keyword_dict = file[file.files[0]].item()

prevalence_df = pd.read_csv('../data/matches_on_words_df{}.csv'.format('2022-06-13'), index_col=0)

In [139]:
def get_locations_of_keywords(text_file=text_dict['google']['google_2020_Q2']['text'],
                             category ='DBS'):
    locations_of_keywords = {'words':[],'starts' : [], 'stops': []}
    for index in keyword_dict[category]:
        #for form in ['text','lemmas','stems']:
        p = re.compile(keyword_dict[category][index]['text'])
        for m in p.finditer(text_file):
            locations_of_keywords["words"].append(m.group())
            locations_of_keywords["starts"].append(m.start())
            locations_of_keywords["stops"].append(m.end())
    return locations_of_keywords

def prevalence(keyword):
    return list(prevalence_df[prevalence_df['index'] == keyword]['count'])[0]/prevalence_df["count"].max()
    
def get_distances_and_prevalence(text_file=text_dict['google']['google_2020_Q2']['text']):
    '''a list of distances from the start and end of each scope keyword to the nearest dbs keyword'''
    distances_and_prevalence = {}
    dbs = get_locations_of_keywords(category=list(keyword_dict.keys())[0])

    for category in keyword_dict.keys():
        if category == 'DBS':
            pass
        else:
            distances_and_prevalence[category] = {}
            locs = get_locations_of_keywords(text_file, category=category)
            distance_to_dbs_keywords = [[min(abs(np.array(dbs['starts']+dbs['stops'])-i)), min(abs(np.array(dbs['starts']+dbs['stops'])-j)) ] for i, j in zip(locs['starts'], locs['stops'])]
            distances_and_prevalence[category]["distances"] = np.min(np.array(distance_to_dbs_keywords), axis=1)
            distances_and_prevalence[category]["prevalence"] = [prevalence(word) for word in locs['words']]
    return distances_and_prevalence

In [140]:
get_distances_and_prevalence()

{'Scope of Digital Business Strategy': {'distances': array([ 97,  72,   1, 642, 625, 694, 580, 382, 298,  77, 208, 231, 212,
         417, 523, 176,   1, 642, 625, 694, 580, 382, 298,  77, 208, 231,
         212, 417, 523, 139, 330, 575]),
  'prevalence': [0.23321917808219178,
   0.23321917808219178,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.22910958904109588,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.5061643835616438,
   0.14726027397260275,
   0.43767123287671234]},
 'S

# big question: *How do we convert this to a score?*
**positiv correlation with amount of keywords**
* sum?

**negativ correlation with distances**
* exponential decay

**negativ correlation with keyword prevelance in entire set**
* multiply by percentage of reports not containing keyword, ie 1 minus prevelance: $1-P({k})$

$$
S = \sum_\text{$k$ in keywordMatches} \left( e^{-a(d-1)}\cdot(1-P(k))\right)
$$

**key:**
* $S$ : Score
* $k$ : keyword
* $P$ : keyword prevalence
* $a$ : factor on decay speed
* $d$ : distance to DBS keyword




In [124]:
def getScore(distances_and_prevalence, a=.5):
    score = 0
    for category in distances_and_prevalence.keys():
        arr = np.array(list(distances_and_prevalence[category].values()))
        arr[0] = np.exp(-a*(arr[0]-1))
        arr[1] = 1-arr[1]
        score += sum(arr[0]*arr[1])
    return score

# Now lets get the score for all the shareholder decks

In [143]:
for report in text_dict['google'].keys():
    print(report, getScore(get_distances_and_prevalence(text_dict['google'][report]['text'])))

google_2020_Q4 5.767153246927597
google_2018_Q1 7.039783123362155
google_2018_Q3 4.16011372268695
google_2018_Q2 3.837305191898948
google_2020_Q2 1.6286732335467036
google_2020_Q3 4.3777021418959645
google_2020_Q1 3.5303140531125754
google_2018_Q4 7.246624122986973
google_2019_Q2 4.236504611499832
google_2019_Q3 4.953110837903881
google_2019_Q1 7.894337069552871
google_2021_Q4 5.884619786619539
google_2019_Q4 2.902820262463728
google_2021_Q1 8.495371330414383
google_2021_Q3 8.49547217148502
google_2021_Q2 3.9475864797167666


In [164]:
data = {}
i = 0
for company in text_dict.keys():
    if i<5:
        data[company] = {}
        for report in text_dict[company].keys():
            try:
                report_formatted = ''.join(report.split('_')[1:])
                data[company][report_formatted] = getScore(get_distances_and_prevalence(text_dict[company][report]['text']))
            except:
                print(report)

    i += 1       
pd.DataFrame.from_dict(data).sort_index()

apple_2017_Q2
moderna_2022_Q1


,apple,paccar,starbucks,moderna,honeywell
2017Q1,0.045815,NaN,2.994205,NaN,1.236185
2017Q2,NaN,NaN,1.979597,NaN,0.341239
2017Q3,1.762172,NaN,5.019460,NaN,2.225066
2017Q4,0.827190,NaN,3.634606,NaN,2.277524
2018Q1,1.011644,NaN,2.171051,NaN,4.849217
2018Q2,0.362945,NaN,3.367722,NaN,4.984836
2018Q3,0.558414,NaN,4.247086,NaN,4.773709
2018Q4,1.625282,NaN,6.226827,0.876951,1.224487
2019Q1,1.299915,NaN,1.945632,2.051170,1.750490
2019Q2,1.669520,NaN,4.508376,0.425605,0.306680


,apple,paccar,starbucks,moderna,honeywell
2020Q4,1.028349,5.257499,5.157410,3.061365,1.038047
2022Q1,0.202791,1.179227,8.748137,NaN,0.602907
2020Q3,1.055516,NaN,4.313471,2.038580,2.024903
2020Q2,0.584305,NaN,4.313471,0.841160,2.919194
2020Q1,2.094207,NaN,4.169138,4.196209,0.994633
2019Q3,1.281028,NaN,4.280781,0.668218,0.126574
2019Q2,1.669520,NaN,4.508376,0.425605,0.306680
2021Q4,0.352916,1.277661,2.837307,3.658519,2.146858
2019Q1,1.299915,NaN,1.945632,2.051170,1.750490
2021Q1,4.824927,2.967633,6.773552,2.756257,0.994633
